In [8]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from chatbot_test import ChatBotTs, load_api_key
from OktSentiment import Sentiment

app = Flask(__name__)
CORS(app)

# ChatBot 인스턴스 생성
api_key = load_api_key()
if not api_key:
    raise ValueError("API key not found or invalid. Please check your api_key.txt file.")
chatbot = ChatBotTs(api_key, 'C:\\Users\\SMHRD\\Documents\\카카오톡 받은 파일\\원인-대처종합.csv')  # CSV 파일 경로를 적절히 수정하세요
#C:\Users\SMHRD\Documents\카카오톡 받은 파일\원인-대처종합.csv //c:/Users/SMHRD/원인-대처종합.csv
# Sentiment 인스턴스 생성
sentiment_analyzer = Sentiment()

@app.route("/chatbot", methods=["POST"])
def chat():
    data = request.json
    print("Received data:", data)
    user_nick = data.get("user_nick")
    user_input = data.get("input")
    
    if user_input:
        try:
            response = chatbot.process_input(user_nick, user_input)
            if response.get("isProblem"):
                # 형태소 분석 및 감정 분석 수행
                sentiment_results = sentiment_analyzer.parse([user_input])
                # 결과가 딕셔너리 형태로 바꾸고 소수점 소수점 둘쨰자리까지
                if 'polarity' in sentiment_results and len(sentiment_results['polarity']) > 0:
                    sentiment_results = sentiment_results['polarity'][0]
                    sentiment_results = {key: round(value * 100, 2) for key, value in sentiment_results.items()}

                response.update({
                    "sentiment": sentiment_results
                })

            return jsonify(response)
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            return jsonify({"error": f"An error occurred while processing your request: {str(e)}"}), 500
    else:
        return jsonify({"error": "No input provided"}), 400

if __name__ == "__main__":
    app.run(port=7000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:7000
Press CTRL+C to quit
127.0.0.1 - - [02/Sep/2024 16:07:12] "OPTIONS /chatbot HTTP/1.1" 200 -


Received data: {'user_nick': '12', 'input': '요즘 고민이 너무 많아요'}
Processing input: user_nick=12, user_input=요즘 고민이 너무 많아요
Is problem flag: True
고민임
Extracted reason: '정신건강'
Problem reason: 정신건강


127.0.0.1 - - [02/Sep/2024 16:07:14] "POST /chatbot HTTP/1.1" 500 -


Extracted reason: '일반고민'
Problem reason: 일반고민
정렬된 형태소 분석 결과: ['요/즘', 'N/o']
정렬된 형태소 분석 결과: ['고/민', 'N/o']
Error occurred: string index out of range
